In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from vgg19 import build_vgg19_unet
from metrics1 import dice_loss, dice_coef

In [2]:
""" Global Parameters """
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, r"C:\Users\User\Desktop\cubaan2\data\images", "*.png")))
    masks = sorted(glob(os.path.join(path, r"C:\Users\User\Desktop\cubaan2\data\mask", "*.png")))

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=4):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(10)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("filesV19")

    """ Hyperparameters """
    batch_size = 12
    lr = 0.01
    num_epochs = 5
    model_path = os.path.join("filesV19", "modelvggt1.h5")
    csv_path = os.path.join("filesV19", "datavggt1.csv")

    """ Dataset """
    dataset_path = r"C:\Users\User\Desktop\cubaan2\data"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
    train_x, train_y = shuffling(train_x, train_y)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

    train_steps = len(train_dataset)
    valid_steps = len(valid_dataset)

    """ Model """
    model = build_vgg19_unet((H, W, 3))
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef])

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Train: 1376 - 1376
Valid: 458 - 458
Test: 458 - 458
Epoch 1/5
115/115 [==============================] - ETA: 0s - loss: 0.0358 - dice_coef: 0.9642 
Epoch 1: val_loss improved from inf to 0.19806, saving model to filesV19\modelvggt1.h5
115/115 [==============================] - 3029s 26s/step - loss: 0.0358 - dice_coef: 0.9642 - val_loss: 0.1981 - val_dice_coef: 0.8027 - lr: 0.0100
Epoch 2/5
115/115 [==============================] - ETA: 0s - loss: 0.0212 - dice_coef: 0.9788 
Epoch 2: val_loss improved from 0.19806 to 0.18195, saving model to filesV19\modelvggt1.h5
115/115 [==============================] - 3033s 26s/step - loss: 0.0212 - dice_coef: 0.9788 - val_loss: 0.1820 - val_dice_coef: 0.8188 - lr: 0.0100
Epoch 3/5
115/115 [==============================] - ETA: 0s - loss: 0.0188 - dice_coef: 0.9812 
Epoch 3: val_loss improved from 0.18195 to 0.01402, saving model to filesV19\modelvggt1.h5
115/115 [==============================] - 3025s 26s/step - loss: 0.0188 - dice_coef: 0.98